# Problems

[Try this yourself](https://colab.research.google.com/github/DTUWindEnergy/TopFarm2/blob/master/docs/notebooks/drivers.ipynb) (requires google account)

**Jump to example in this notebook**:

* [Turbine location optimization](#Turbine-location-optimization)

Make sure you first run the code below in order to set up and initialize needed variables.

In [ ]:
%%capture
try:  # install Topfarm if needed
    import topfarm
except ModuleNotFoundError:
    !pip install topfarm

In [ ]:
import matplotlib.pyplot as plt 

from topfarm import TopFarmProblem
from topfarm.easy_drivers import EasyScipyOptimizeDriver
from topfarm.examples.iea37 import get_iea37_initial, get_iea37_constraints, get_iea37_cost
from topfarm.plotting import NoPlot, XYPlotComp

In [ ]:
# non-updating, inline plots
%matplotlib inline
# ...or updating plots in new window
# %matplotlib qt

## Turbine location optimization

This example optimizes the locations of the 9-turbine benchmark wind farm from IEA Task 37 using the provided initial locations and the `EasyScipyOptimizeDriver`. Details on the benchmark can be found in the following reference:  

* Baker et al. (2019) "Best Practices for Wake Model and Optimization Algorithm Selection in Wind Farm Layout Optimization". AIAA 2019.

In [ ]:
def optimize_iea37_locs(n_wt, driver):
    initial = get_iea37_initial(n_wt)
    tf = TopFarmProblem(
            dict(zip('xy', (initial[:, :2]).T)),
            get_iea37_cost(n_wt),
            constraints=get_iea37_constraints(n_wt),
            driver=driver,
            plot_comp=XYPlotComp())
    tf.optimize()

In [ ]:
optimize_iea37_locs(9, EasyScipyOptimizeDriver(disp=False))

The final optimized output is much lower than the value reported in Baker et al. (2019), which is 257.790 GWh. Moreover, the layout does not match the figures given in Appendix A in the same reference. This is due to the fact that the SLSQP optimizer was attracted to a local minimum. To find the global optimum, more advanced optimization procedures should be used. This benchmark is discussed in more detail in the validation report linked in TOPFARM's documentation.